In [46]:
# After having multiple issues when importing the comments.csv in SQL, we went 'a little' deeper with the cleaning...

In [47]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pandas import json_normalize
import re
import numpy as np

In [48]:
comments = pd.read_csv('comments_draft.csv')

In [49]:
comments

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,createdAt,program,overallScore,overall,curriculum,jobSupport,review_body,school,school_id
0,306683,Pichamon Alvarez,False,NaN,2023.0,False,NaN,Data Analytics Career Track,11/24/2023,Data Analytics Career Track,5.0,5.0,5.0,5.0,Springboard's Data Analytics Career Track not ...,springboard,11035
1,306667,Ethan Reynolds,False,NaN,2023.0,False,NaN,Excellent Experience with Springboard!,11/23/2023,Software Engineering Career Track,4.3,4.0,5.0,4.0,Overall I had a great experience going through...,springboard,11035
2,306638,Constance Sun,False,NaN,2023.0,False,NaN,Very accommodating. Take Springboard if you wa...,11/21/2023,UI/UX Design Career Track,3.7,4.0,4.0,3.0,When I was looking at which bootcamp to do for...,springboard,11035
3,306549,Daniel Dluzynski,False,NaN,2023.0,False,NaN,Extensive and well built curriculum,11/17/2023,Cyber Security Career Track,4.3,4.0,5.0,4.0,This course is great for beginners. The curric...,springboard,11035
4,306505,Jonathan Chiu,False,NaN,2023.0,False,NaN,Join if you're looking to structure &amp; Netw...,11/15/2023,UI/UX Design Career Track,4.0,4.0,4.0,4.0,"If you find yourself unsure of where to begin,...",springboard,11035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,231691,Abby Howell,False,NaN,2013.0,True,Software Developer at Cengage Learning,From 2nd grade teacher to full-stack web devel...,6/29/2014,Software Engineering Immersive,5.0,5.0,5.0,5.0,My experience at General Assembly's Web Develo...,general-assembly,10761
6987,231827,Anonymous,False,NaN,NaN,False,NaN,What you get out of the program really depends...,6/15/2014,NaN,3.0,3.0,NaN,NaN,What you get out of the program really depends...,general-assembly,10761
6988,231816,Thomas Berry,False,NaN,NaN,False,NaN,The bitmaker program provides opportunities an...,6/15/2014,NaN,5.0,5.0,NaN,NaN,Personally I had a great experience at Bitmake...,general-assembly,10761
6989,231836,Ryan Racioppo,False,NaN,NaN,False,NaN,Bitmaker is the best way to motivate and accel...,6/15/2014,NaN,5.0,5.0,NaN,NaN,I was in the 3rd cohort and have had a success...,general-assembly,10761


In [50]:
# transforming years from float to integers - again, as converting the first df draft to csv and reading it in pandas seemed to have changed the data type.
comments['graduatingYear'] = comments['graduatingYear'].astype('Int64')
comments.graduatingYear.value_counts()

2018    1466
2019    1106
2020    1009
2017     964
2021     768
2022     616
2016     421
2023     330
2015     183
2014      47
2012      13
2013      10
2011       1
Name: graduatingYear, dtype: Int64

In [51]:
comments.isna().sum()

id                    0
name                  0
anonymous             0
hostProgramName    4024
graduatingYear       57
isAlumni              1
jobTitle           4119
tagline               4
createdAt             0
program            1900
overallScore          9
overall               9
curriculum          176
jobSupport          791
review_body           0
school                0
school_id             0
dtype: int64

In [52]:
comments.dtypes

id                   int64
name                object
anonymous             bool
hostProgramName     object
graduatingYear       Int64
isAlumni            object
jobTitle            object
tagline             object
createdAt           object
program             object
overallScore       float64
overall            float64
curriculum         float64
jobSupport         float64
review_body         object
school              object
school_id            int64
dtype: object

In [53]:
comments.dropna(subset=['overallScore'], inplace=True)

In [54]:
comments.dropna(subset=['curriculum'], inplace=True)

In [55]:
comments.dropna(subset=['jobSupport'], inplace=True)

In [56]:
comments.drop(998, inplace=True)

In [57]:
comments['overallScore'] = pd.to_numeric(comments['overallScore'], errors='coerce')
comments['overallScore'].dropna(inplace=True)

In [58]:
comments['hostProgramName'].fillna('Not specified', inplace=True)

In [59]:
comments['jobTitle'].fillna('Not specified', inplace=True)

In [60]:
comments.dropna(subset='graduatingYear', inplace=True)

In [61]:
comments['curriculum'] = pd.to_numeric(comments['curriculum'], errors='coerce')

In [62]:
comments['jobSupport'] = pd.to_numeric(comments['jobSupport'], errors='coerce')

In [63]:
comments.columns

Index(['id', 'name', 'anonymous', 'hostProgramName', 'graduatingYear',
       'isAlumni', 'jobTitle', 'tagline', 'createdAt', 'program',
       'overallScore', 'overall', 'curriculum', 'jobSupport', 'review_body',
       'school', 'school_id'],
      dtype='object')

In [64]:
comments.dtypes

id                   int64
name                object
anonymous             bool
hostProgramName     object
graduatingYear       Int64
isAlumni            object
jobTitle            object
tagline             object
createdAt           object
program             object
overallScore       float64
overall            float64
curriculum         float64
jobSupport         float64
review_body         object
school              object
school_id            int64
dtype: object

In [65]:
comments.drop(columns={'anonymous', 'overall', 'program', 'createdAt'}, inplace=True)

In [66]:
comments.columns

Index(['id', 'name', 'hostProgramName', 'graduatingYear', 'isAlumni',
       'jobTitle', 'tagline', 'overallScore', 'curriculum', 'jobSupport',
       'review_body', 'school', 'school_id'],
      dtype='object')

In [67]:
comments.loc[comments['jobTitle'] == '', 'jobTitle'] = 'Not specified'

In [68]:
comments.dropna(subset='jobTitle', inplace=True)

In [69]:
comments.isnull().sum()

id                 0
name               0
hostProgramName    0
graduatingYear     0
isAlumni           0
jobTitle           0
tagline            0
overallScore       0
curriculum         0
jobSupport         0
review_body        0
school             0
school_id          0
dtype: int64

In [70]:
comments.drop(columns='name', inplace=True)

In [71]:
comments.drop(columns='school', inplace=True)

In [72]:
# Attempts to eliminate special charchters from object type columns: WIP

In [73]:
# Trying to troubleshoot the huge loss of data when importing comments

In [74]:
# Check for any missing values in the DataFrame
print("Missing values:\n", comments.isnull().sum())

Missing values:
 id                 0
hostProgramName    0
graduatingYear     0
isAlumni           0
jobTitle           0
tagline            0
overallScore       0
curriculum         0
jobSupport         0
review_body        0
school_id          0
dtype: int64


In [75]:
# Check for unique values in id column
comments.id.value_counts(ascending=False)

306683    1
256926    1
256973    1
256974    1
256975    1
         ..
267640    1
267742    1
267864    1
267895    1
231691    1
Name: id, Length: 6177, dtype: int64

In [76]:
# Check for the presence of special characters or unexpected data
for column in comments.columns:
    # Skip numeric columns
    if pd.api.types.is_numeric_dtype(comments[column]):
        continue

    special_characters = comments[comments[column].astype(str).str.contains(r'[^a-zA-Z0-9\s]', na=False)]
    if not special_characters.empty:
        print(f"Special characters in {column}:\n{special_characters}")

Special characters in hostProgramName:
          id hostProgramName  graduatingYear isAlumni                jobTitle  \
950   267361    UX/UI Design            2020    False           Not specified   
951   267344    UX/UI Design            2020    False  UX designer/consultant   
957   267000    UX/UI Design            2020    False    UX Research/Designer   
958   266956    UX/UI Design            2020    False         UX/ UI Designer   
959   266952    UX/UI Design            2020    False           Not specified   
...      ...             ...             ...      ...                     ...   
6892  236669    UX/UI Design            2016     True     Creative Specialist   
6893  244743    UX/UI Design            2015     True        Product Designer   
6896  244731    UX/UI Design            2016     True             UX Designer   
6916  244545    UX/UI Design            2016     True             UX Designer   
6973  232579    UX/UI Design            2015     True           Not sp

In [77]:
#It seems that the problem in importing is just that there are too many special charachters and the rows containing them are automatically dropped.

In [78]:
special_characters = r'[!@#$%^&*()_+{}|:"<>?\\[\];,./\']'

# Identify string columns
string_columns = comments.select_dtypes(include='object').columns

for columns in string_columns:
    comments[column] = comments[column].astype(str).apply(lambda x: x.replace(special_characters, ''))

# checking
comments

,id,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,overallScore,curriculum,jobSupport,review_body,school_id
0,306683,Not specified,2023,False,Not specified,Data Analytics Career Track,5.0,5.0,5.0,Springboard's Data Analytics Career Track not ...,11035
1,306667,Not specified,2023,False,Not specified,Excellent Experience with Springboard!,4.3,5.0,4.0,Overall I had a great experience going through...,11035
2,306638,Not specified,2023,False,Not specified,Very accommodating. Take Springboard if you wa...,3.7,4.0,3.0,When I was looking at which bootcamp to do for...,11035
3,306549,Not specified,2023,False,Not specified,Extensive and well built curriculum,4.3,5.0,4.0,This course is great for beginners. The curric...,11035
4,306505,Not specified,2023,False,Not specified,Join if you're looking to structure &amp; Netw...,4.0,4.0,4.0,"If you find yourself unsure of where to begin,...",11035
...,...,...,...,...,...,...,...,...,...,...,...
6973,232579,UX/UI Design,2015,True,Not specified,GA is great on most levels,4.0,4.0,4.0,"Pros: Great environment, phenomenal network, a...",10761
6974,232573,Not specified,2015,True,Not specified,"General Assembly UXDI an amazing experience, i...",4.3,4.0,4.0,I was beyond impressed by my experience at Gen...,10761
6978,232490,Not specified,2015,True,Not specified,Incredible Learning Opportunity for Digital Ma...,5.0,5.0,5.0,I recently completed an Ad Optimization part t...,10761
6983,232206,Not specified,2014,True,Not specified,Not The Best Bootcamp I Could Have Chosen,1.7,2.0,1.0,I wouldn't say that Bitmaker was a bad experie...,10761


In [79]:
# check for duplicates to verify SQL is not dropping them in the upload and therefore losing rows.
duplicates = comments[comments.duplicated()]
if not duplicates.empty:
    print("Duplicate rows:\n", duplicates)

In [80]:
# The code does not get error, but the importing issues in SQL are persisting.
# In our lack of time we find a compromise solution: we export of filtered comments with numerical columns to perform our queries as object columns with all text information were not necessary.
# It would have nice nice though to solve the issue and have the complete dataframe comments uploaded in SQL.
# Without these columns we were able to use almost all rows in the df.

In [81]:
# Identify string columns
string_columns = comments.select_dtypes(include='object').columns
string_columns

Index(['hostProgramName', 'isAlumni', 'jobTitle', 'tagline', 'review_body',
       'school_id'],
      dtype='object')

In [82]:
comments_numeric = comments.drop(columns={'hostProgramName', 'isAlumni', 'jobTitle', 'tagline', 'review_body'})

In [83]:
comments_numeric

,id,graduatingYear,overallScore,curriculum,jobSupport,school_id
0,306683,2023,5.0,5.0,5.0,11035
1,306667,2023,4.3,5.0,4.0,11035
2,306638,2023,3.7,4.0,3.0,11035
3,306549,2023,4.3,5.0,4.0,11035
4,306505,2023,4.0,4.0,4.0,11035
...,...,...,...,...,...,...
6973,232579,2015,4.0,4.0,4.0,10761
6974,232573,2015,4.3,4.0,4.0,10761
6978,232490,2015,5.0,5.0,5.0,10761
6983,232206,2014,1.7,2.0,1.0,10761


In [84]:
comments_numeric.to_csv('comments_numeric.csv', index=False)
comments.to_csv('comments_cleaned_not_for_SQL.csv', index=False)